In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
data = pd.read_csv('../input/real-time-advertisers-auction/Dataset.csv')
DATE = '2019-06-21'

In [ ]:
data.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

data['CPM'] = data.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
data = data[data['CPM'] >= 0]
train = data[data['date'] <= DATE]
test = data[data['date'] > DATE]
train = train[train['CPM'] < train['CPM'].quantile(.95)]
test = test[test['CPM'] < test['CPM'].quantile(.95)]

X_train = train.drop(['CPM', 'date', 'total_revenue'], axis=1)
y_train = train['CPM'].values
X_test = test.drop(['CPM', 'date', 'total_revenue'], axis=1)
y_test = test['CPM'].values

In [ ]:
X_train.info()

In [ ]:
X_test.info()

In [ ]:
X_train.columns

In [ ]:
cat_cols = []
for col in X_train.columns:
    if 'id' in col:
        cat_cols.append(col)
print(cat_cols)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=False, random_state=17)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols, free_raw_data=False)
val_data =  lgb.Dataset(X_val, label=y_val, categorical_feature=cat_cols, free_raw_data=False) 

In [ ]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    "num_leaves": 31, 
    
}
num_round = 200
bst = lgb.train(param, train_data, num_round, valid_sets=[val_data])

In [ ]:
y_pred = bst.predict(X_test)
mean_squared_error(y_test, y_pred)